# ETL
- The purpose of this notebook is to design The ETL process for Data Modeling with postgres project

In [1]:
import pandas as pd
import os

import psycopg2
from sql_quries import *

In [2]:
#start the connection with database
try:
    conn  = psycopg2.connect("dbname = sparkifydb user = postgres password=")
except psycopg2.Error as e:
    print("Couldnot connect to database")
    print(e)

In [3]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Couldnot get cursor")
    print(e)

In [4]:
conn.set_session(autocommit = True)

In [5]:
log_files = []
song_files = []
for (root,dirs,files) in os.walk('data', topdown=True):
        if len(files) !=0:
            if 'log_data' in root:
                for i in files:
                    log_files.append(os.path.join(root , i))
                    
            if 'song_data' in root:
                for i in files:
                    song_files.append(os.path.join(root , i))
                
            

In [6]:
df = pd.read_json(song_files[2] , lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0


In [7]:
df = pd.read_json(log_files[2] , lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,52,None,307,1541207073796,None,
1,None,Logged In,Celeste,F,1,Williams,NaN,free,"Klamath Falls, OR",GET,Home,1.541078e+12,52,None,200,1541207123796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
2,Mynt,Logged In,Celeste,F,2,Williams,166.94812,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Playa Haters,200,1541207150796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Taylor Swift,Logged In,Celeste,F,3,Williams,230.47791,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,You Belong With Me,200,1541207316796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,Amy Winehouse,Logged In,Celeste,F,4,Williams,229.85098,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Valerie,200,1541207546796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53


# ETL to songs Table
- columns : song_id, title, artist_id, year, duration

In [8]:
df = pd.read_json(song_files[0] , lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [9]:
song_data = df[['song_id' , 'title' , 'artist_id' , 'year' , 'duration']].values[0].tolist()
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

In [10]:
insert_into_songs(cur , tuple(song_data))

In [11]:
select_all(cur , 'songs')

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, Decimal('218.93179'))


# ETL to artists Table
- columns:artist_id, name, location, latitude, longitude

In [12]:
artist_data = df[['artist_id' , 'artist_name' , 'artist_location' ,'artist_latitude' ,'artist_longitude']].values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

In [13]:
insert_into_artists(cur , tuple(artist_data))

In [14]:
select_all(cur , 'artists')

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', Decimal('NaN'), Decimal('NaN'))


# Process log data
- In this section I'll process the log data to create users and time dimensions tables

In [15]:
df = pd.read_json(log_files[0] , lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


# ETL to time Table
- columns: start_time, hour, day, week, month, year, weekday

In [16]:
df = df[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [17]:
df['ts'] = pd.to_datetime(df['ts'] , unit = 'ms')
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,2018-11-01 21:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,2018-11-01 21:08:16.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,2018-11-01 21:11:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,2018-11-01 21:17:33.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [18]:
time_data = (df['ts'] , df['ts'].dt.hour ,df['ts'].dt.day , \
             df['ts'].dt.isocalendar().week, df['ts'].dt.month, df['ts'].dt.year,df['ts'].dt.weekday )
cols_names = ('start_time' , 'hour' , 'day' , 'week' , 'month' , 'year' , 'weekday')

In [19]:
time_df = pd.DataFrame(columns = cols_names)
for idx , col_name in enumerate(cols_names):
    time_df[col_name] = time_data[idx]

In [20]:
time_df.head()

,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796,21,1,44,11,2018,3
5,2018-11-01 21:08:16.796,21,1,44,11,2018,3
6,2018-11-01 21:11:13.796,21,1,44,11,2018,3
7,2018-11-01 21:17:33.796,21,1,44,11,2018,3


In [21]:
for _ , row in time_df.iterrows():
    
    insert_into_time(cur , tuple(row.tolist()))

In [22]:
select_all(cur , 'time')

(datetime.datetime(2018, 11, 1, 21, 1, 46, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 5, 52, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 8, 16, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 11, 13, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 17, 33, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 24, 53, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 28, 54, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 42, 0, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 52, 5, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 55, 25, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 22, 23, 14, 796000), 22, 1, 44, 11, 2018, 3)


# ETL to users Table
- user_id, first_name, last_name, gender, level

In [23]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,2018-11-01 21:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,2018-11-01 21:08:16.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,2018-11-01 21:11:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,2018-11-01 21:17:33.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [24]:
user_data = df[['userId' , 'firstName' , 'lastName' , 'gender' ,'level']]
user_data

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free
8,8,Kaylee,Summers,F,free
9,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
13,26,Ryan,Smith,M,free


In [25]:
for _ , row in user_data.iterrows():
    insert_into_users(cur , tuple(row.tolist()))

In [26]:
select_all(cur , 'users')

(8, 'Kaylee', 'Summers', 'F', 'free')
(10, 'Sylvie', 'Cruz', 'F', 'free')
(26, 'Ryan', 'Smith', 'M', 'free')
(101, 'Jayden', 'Fox', 'M', 'free')


# ETL to songplays Table
- columns: songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

In [27]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,2018-11-01 21:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,2018-11-01 21:08:16.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,2018-11-01 21:11:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,2018-11-01 21:17:33.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


- Everey column in songplays table will be extracted from log files except song_id , artist_id

In [28]:
for _ , row in df.iterrows():
    song_artist_ids = song_select(cur , row['song'] , row['length'] , row['artist'])
    
    if song_artist_ids:
        #extract song_id & artist_id from songs and artists tables
        song_id , artist_id = song_artist_ids
    else:

        song_id , artist_id = None , None
    
    values = (row['ts'] , row['userId'] , row['level'] ,song_id ,artist_id,row['sessionId'] , row['location'] , row['userAgent'])
    insert_into_songplays(cur , values)
    
    

In [29]:
select_all(cur , 'songplays')

(1, datetime.datetime(2018, 11, 1, 21, 1, 46, 796000), 8, 'free', None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"')
(2, datetime.datetime(2018, 11, 1, 21, 5, 52, 796000), 8, 'free', None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"')
(3, datetime.datetime(2018, 11, 1, 21, 8, 16, 796000), 8, 'free', None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"')
(4, datetime.datetime(2018, 11, 1, 21, 11, 13, 796000), 8, 'free', None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"')
(5, datetime.datetime(2018, 11, 1, 21, 17, 33, 796000), 8, 'free', None, None, 139,

In [30]:
cur.close()
conn.close()